In [1]:
# هذا السطر يفتح اتصالاً واحداً مستقراً وسريعاً
segmenter = FarasaSegmenter(interactive=True)

NameError: name 'FarasaSegmenter' is not defined

In [ ]:
# ===============================
# مشروع NLP عربي - Farasa محسن
# ===============================

# 1️⃣ استيراد المكتبات
import warnings
warnings.filterwarnings("ignore")  # إخفاء التحذيرات

from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from farasa.pos import FarasaPOSTagger
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity

# 2️⃣ بيانات عربية
texts = [
    "والطلاب يدرسون اللغة العربية في الجامعة",
    "الكتابات العلمية مهمة للبحث والتعلم",
    "أحب قراءة الكتب وتوسيع المعرفة",
    "المدرسة تنظم الأنشطة الطلابية بشكل مستمر",
    "بحث الطلاب عن المعلومات مفيد ومهم",
    "أحب العلم والتعلم المستمر يوميًا",
    "الطلاب يشاركون في المسابقات العلمية",
    "الكتب تحتوي على معلومات قيمة للطلاب",
    "الدراسة تساعد في النجاح والتقدم",
    "المعرفة مفتاح التقدم والتطور"
]

labels = [
    "تعليم", "بحث", "ثقافة", "تعليم", "بحث",
    "ثقافة", "تعليم", "ثقافة", "تعليم", "ثقافة"
]

# 3️⃣ تهيئة Farasa
segmenter = FarasaSegmenter()
stemmer = FarasaStemmer()
pos_tagger = FarasaPOSTagger()

# 4️⃣ معالجة النصوص: التقطيع + الجذور + POS
processed_texts = []
pos_texts = []

for text in texts:
    segmented = segmenter.segment(text)       # التقطيع
    stemmed = stemmer.stem(segmented)         # استخراج الجذور
    pos_tags = pos_tagger.tag(text)           # تصنيف نحوي
    processed_texts.append(stemmed)
    pos_texts.append(pos_tags)

print("=== نصوص بعد التقطيع والجذور ===")
for original, processed, pos in zip(texts, processed_texts, pos_texts):
    print(f"\nالنص الأصلي: {original}")
    print(f"بعد التقطيع والجذور: {processed}")
    print(f"POS Tags: {pos}")

# 5️⃣ تحويل النصوص إلى TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(processed_texts)
y = labels

# 6️⃣ تقسيم البيانات وتدريب نموذج تصنيف نصوص
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
model = MultinomialNB()
model.fit(X_train, y_train)

# 7️⃣ تقييم النموذج
y_pred = model.predict(X_test)
print("\n=== تقييم نموذج التصنيف ===")
print("الدقة:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

# 8️⃣ وظيفة البحث النصي
def search_text(query, top_n=3):
    segmented = segmenter.segment(query)
    stemmed = stemmer.stem(segmented)
    query_vec = vectorizer.transform([stemmed])
    similarity = cosine_similarity(query_vec, X)
    results = sorted(
        [(idx, score) for idx, score in enumerate(similarity[0])],
        key=lambda x: x[1],
        reverse=True
    )
    print(f"\n=== نتائج البحث عن: '{query}' ===")
    for idx, score in results[:top_n]:
        print(f"النص: {texts[idx]} | درجة التشابه: {score:.3f}")

# 9️⃣ وظيفة تصنيف نصوص جديدة
def classify_text(query):
    segmented = segmenter.segment(query)
    stemmed = stemmer.stem(segmented)
    query_vec = vectorizer.transform([stemmed])
    prediction = model.predict(query_vec)
    print(f"\nالنص: {query} --> التصنيف المتوقع: {prediction[0]}")

# أمثلة على الاستخدام
search_text("طلاب")
classify_text("أحب القراءة وتعلم العلوم")


C:\ProgramData\anaconda66\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


  4%|███▏                                                                       | 10.2M/241M [05:20<1:49:34, 35.1kiB/s]